In [2]:
import os  # To interact with the file system
import numpy as np  # For numerical operations
import tensorflow as tf  # For deep learning functionality
from tensorflow.keras.preprocessing.image import ImageDataGenerator # For image data preprocessing
import matplotlib.pyplot as plt  # For visualizing images and data


In [3]:
# Set the directories for training and testing images
train_dir = 'D:/Stroke-Prediction-Using-Medical-Imaging/data/training'
test_dir = 'D:/Stroke-Prediction-Using-Medical-Imaging/data/testing'

# Define the image size to which we will resize the images
img_size = (256, 256)  # This is a standard size for many deep learning models (VGG16, etc.)


In [4]:
# Initialize ImageDataGenerator for training data with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Rescaling the images (normalize pixel values to the range [0, 1])
    rotation_range=20,  # Random rotations for data augmentation
    width_shift_range=0.2,  # Horizontal shifts for data augmentation
    height_shift_range=0.2,  # Vertical shifts for data augmentation
    shear_range=0.2,  # Shear transformations
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Flip images horizontally for data augmentation
    fill_mode='nearest'  # Fill empty pixels after transformations
)

# Initialize ImageDataGenerator for testing data with only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)  # Only rescaling for test data


In [5]:
# Load training data from the directory, specifying the target image size and batch size
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical', # Since this is a binary classification problem (stroke or no stroke)
    color_mode='grayscale'  # Process images as grayscale
)

# Load testing data
test_generator = test_datagen.flow_from_directory(
    test_dir,  # Path to testing directory
    target_size=(256, 256), # Resize all images to the same size
    batch_size=32,  # Number osizef images to process in each batch
    class_mode='categorical',  # Use 'categorical' since there are 4 classes
    color_mode='grayscale'  # Process images as grayscale
)

Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Input


In [7]:
from tensorflow.keras.optimizers import Adam
# Initialize a Sequential model
model = Sequential([
    Input(shape=(256, 256, 1)),  # Define input explicitly
    Conv2D(32, (3, 3), activation='relu'),# first layer
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'), # Second layer
    MaxPooling2D(pool_size=(2, 2)),
    GlobalAveragePooling2D(),# Flatten the output
    Dense(128, activation='relu'),# Add Dense layers for classification
    Dropout(0.5), # Prevent overfitting
    Dense(4, activation='softmax') # Output layer with 4 classes
])

# Compile the model
model.compile(
    optimizer= Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Display the model architecture
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,652 (108.02 KB)

 Trainable params: 27,652 (108.02 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
from tensorflow.keras.callbacks import EarlyStopping

# Early stopping to stop training when validation accuracy doesn't improve
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=5,           # Stop training after 5 epochs without improvement
    restore_best_weights=True
)

# Train the model using the training data
history = model.fit(
    train_generator,  # Training data
    epochs=30,  # Number of epochs (can be adjusted)
    validation_data=test_generator,  # Validation data
    verbose=1  # Display training progress
)


d:\Stroke-Prediction-Using-Medical-Imaging\stroke_env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
179/179 ━━━━━━━━━━━━━━━━━━━━ 73s 402ms/step - accuracy: 0.2964 - loss: 1.3287 - val_accuracy: 0.4630 - val_loss: 1.1931
Epoch 2/30
179/179 ━━━━━━━━━━━━━━━━━━━━ 75s 416ms/step - accuracy: 0.5079 - loss: 1.1243 - val_accuracy: 0.4882 - val_loss: 1.1604
Epoch 3/30
179/179 ━━━━━━━━━━━━━━━━━━━━ 80s 447ms/step - accuracy: 0.5487 - loss: 1.0499 - val_accuracy: 0.5614 - val_loss: 1.1098
Epoch 4/30
179/179 ━━━━━━━━━━━━━━━━━━━━ 159s 886ms/step - accuracy: 0.5807 - loss: 1.0292 - val_accuracy: 0.5698 - val_loss: 1.0632
Epoch 5/30
179/179 ━━━━━━━━━━━━━━━━━━━━ 78s 436ms/step - accuracy: 0.6081 - loss: 0.9689 - val_accuracy: 0.5507 - val_loss: 1.1024
Epoch 6/30
179/179 ━━━━━━━━━━━━━━━━━━━━ 77s 429ms/step - accuracy: 0.6445 - loss: 0.9099 - val_accuracy: 0.4851 - val_loss: 1.2550
Epoch 7/30
179/179 ━━━━━━━━━━━━━━━━━━━━ 76s 424ms/step - accuracy: 0.6468 - loss: 0.8847 - val_accuracy: 0.5538 - val_loss: 1.1039
Epoch 8/30
179/179 ━━━━━━━━━━━━━━━━━━━━ 77s 429ms/step - accuracy: 0.6423 - loss: 

In [10]:
# Save the trained model
model.save('brain_stroke_classification_model.keras')
print("Model saved successfully!")


Model saved successfully!


In [11]:
from tensorflow.keras.models import load_model

# Load the model saved in Keras format
model = load_model('brain_stroke_classification_model.keras')
print("Model loaded successfully from the Keras format!")


Model loaded successfully from the Keras format!


d:\Stroke-Prediction-Using-Medical-Imaging\stroke_env\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [14]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Path to the single image you want to test
img_path = 'data/testing/meningioma/Te-me_0014.jpg'

# Load and preprocess the image
img = image.load_img(img_path, target_size=(256, 256), color_mode='grayscale')  # Adjust size and color mode if necessary
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize the image

# Make a prediction
prediction = model.predict(img_array)

# Output the prediction
predicted_class = np.argmax(prediction, axis=1)  # Get the index of the highest probability
print("Predicted class:", predicted_class)

# If you want to map the prediction to class labels
class_labels = ['glioma', 'meningioma', 'notumor', 'pituitary']  # Replace with your actual class labels
print("Predicted class label:", class_labels[predicted_class[0]])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Predicted class: [2]
Predicted class label: notumor
